In [1]:
%set_env XLA_PYTHON_CLIENT_PREALLOCATE False

env: XLA_PYTHON_CLIENT_PREALLOCATE=False


In [2]:
import functools
import importlib
import os

import jax.numpy as jnp
import jax_verify
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [3]:
# Define which sensors you want to use
sensors_to_use = [
    "emg",
    "fp",
    "gcLeft",
    "gcRight",
    "gon",
    "id",
    "ik",
    "ik_offset",
    "imu",
    "jp",
    "markers",
]

base_dir = "datasetcsv_output"

trials = ["01", "02", "03", "04", "05", "06", "07", "08"]

# To store X and y for each trial
X_list = []
y_list = []

# Load each dataset and process independently
for trial in trials:
    print(f"\nProcessing trial: {trial}")
    sensor_data_list = []
    trial_common_headers = None

    # Load each sensor data
    for sensor in sensors_to_use:
        sensor_file = f"{sensor}_treadmill_{trial}_01.csv"
        sensor_path = os.path.join(base_dir, sensor_file)
        if os.path.exists(sensor_path):
            df_sensor = pd.read_csv(sensor_path)
            sensor_data_list.append(df_sensor)

            if trial_common_headers is None:
                trial_common_headers = set(df_sensor["Header"])
            else:
                trial_common_headers.intersection_update(df_sensor["Header"])

    conditions_file = f"conditions_treadmill_{trial}_01.csv"
    conditions_path = os.path.join(base_dir, conditions_file)
    if os.path.exists(conditions_path):
        df_conditions = pd.read_csv(conditions_path)

        if trial_common_headers is not None:
            trial_common_headers.intersection_update(df_conditions["Header"])

    if trial_common_headers and sensor_data_list:
        print(
            f"Trial {trial} has {len(trial_common_headers)} common headers, proceeding with merge."
        )

        filtered_sensor_data = [
            df[df["Header"].isin(trial_common_headers)].sort_values(by="Header")
            for df in sensor_data_list
        ]
        filtered_conditions = df_conditions[
            df_conditions["Header"].isin(trial_common_headers)
        ].sort_values(by="Header")

        # Merge all filtered sensor data for this trial
        merged_sensor_data = pd.concat(filtered_sensor_data, axis=1)

        # Drop duplicated 'Header' columns except for the first one
        merged_sensor_data = merged_sensor_data.loc[
            :, ~merged_sensor_data.columns.duplicated()
        ]

        print(
            f"Number of NaN values in merged_sensor_data before merging with conditions: {merged_sensor_data.isna().sum().sum()}"
        )

        # Merge sensor and conditions data for this trial using an inner join to avoid NaN
        merged_data_trial = pd.merge(
            merged_sensor_data, filtered_conditions, on="Header", how="inner"
        )

        # Drop any remaining NaN values after merging
        if merged_data_trial.isna().sum().sum() > 0:
            print(
                f"NaN values detected in merged data for trial {trial}, dropping rows with NaN."
            )
            merged_data_trial = merged_data_trial.dropna()

        # Extract features (X) and target (y) for the trial
        X_trial = merged_data_trial.drop(columns=["Speed"])
        y_trial = merged_data_trial["Speed"]

        # Append to the list for all trials
        X_list.append(X_trial)
        y_list.append(y_trial)

        # Print the number of NaN values after dropping NaN rows
        print(
            f"Number of NaN values in merged_data for trial {trial} after dropping: {merged_data_trial.isna().sum().sum()}"
        )

        # Print the first few rows of the merged data for inspection
        print(
            f"First few rows of merged data for trial {trial}:\n{merged_data_trial.head()}"
        )

    else:
        print(f"Trial {trial} skipped due to no common headers or missing sensor data.")

# Ensure consistency across trials by keeping only columns that are common across all trials
common_headers = {col for X_trial in X_list for col in X_trial.columns}
for X_trial in X_list:
    common_headers.intersection_update(X_trial.columns)

# Filter X to keep only common columns across all trials
X_list = [X_trial[list(common_headers)] for X_trial in X_list]

# Filter out trials with less than 100 timesteps
X_list = [X_trial for X_trial in X_list if len(X_trial) >= 100]
y_list = [y_trial for y_trial in y_list if len(y_trial) >= 100]

# Print the shape of X and y
print(
    f"Shape of Xs after filtering common headers: {[X_trial.shape for X_trial in X_list]}"
)
print(f"Shape of ys: {[y_trial.shape for y_trial in y_list]}")

# Check for NaN values in X and y
nan_count_X = sum(X_trial.isna().sum().sum() for X_trial in X_list)
nan_count_y = sum(y_trial.isna().sum() for y_trial in y_list)

print(f"Number of NaN values in X: {nan_count_X}")
print(f"Number of NaN values in y: {nan_count_y}")

# Check for infinite values in X and y
inf_count_X = sum(np.isinf(X_trial.values).sum() for X_trial in X_list)
inf_count_y = sum(np.isinf(y_trial).sum() for y_trial in y_list)

print(f"Number of infinite values in X: {inf_count_X}")
print(f"Number of infinite values in y: {inf_count_y}")


Processing trial: 01
Trial 01 has 28612 common headers, proceeding with merge.
Number of NaN values in merged_sensor_data before merging with conditions: 5285765
NaN values detected in merged data for trial 01, dropping rows with NaN.
Number of NaN values in merged_data for trial 01 after dropping: 0
First few rows of merged data for trial 01:
      Header  gastrocmed  tibialisanterior    soleus  vastusmedialis  \
716   15.570   -0.063153         -0.002946  0.224090       -0.010610   
717   15.575   -0.043159         -0.000679  0.562477        0.014191   
718   15.580    0.309805          0.003162  0.501198        0.013440   
2943  26.705   -0.015926         -0.024213  0.038289       -0.000549   
2944  26.710   -0.019323          0.019561  0.046283        0.001949   

      vastuslateralis  rectusfemoris  bicepsfemoris  semitendinosus  gracilis  \
716         -0.000409       0.014828      -0.008367       -0.025644 -0.000704   
717         -0.005238      -0.002204       0.000862       

In [4]:
stack_size = 64


def sliding_window(X_trial):
    slidified = np.lib.stride_tricks.sliding_window_view(X_trial, stack_size, -2)

    return slidified


X = np.concat(
    [
        sliding_window(X_trial)
        for X_trial in X_list
    ]
)
y = np.concat([y_trial[stack_size - 1 :] for y_trial in y_list])

print(f"Final shapes:\n\tX: {X.shape}\n\ty: {y.shape}")

Final shapes:
	X: (30672, 214, 64)
	y: (30672,)


In [5]:
# Convert to NumPy arrays if needed
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Custom Dataset Class
class SpeedDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32)).to(device)
        self.y = torch.from_numpy(y.astype(np.float32)).to(device)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]


# Create Datasets and DataLoaders
train_dataset = SpeedDataset(X_train, y_train)
val_dataset = SpeedDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

In [6]:
class TemporalBlock(nn.Module):
    def __init__(
        self, in_channels, out_channels, kernel_size, stride, dilation, dropout=0.2
    ):
        super(TemporalBlock, self).__init__()
        # Calculate padding to keep output size the same as input size
        padding = (kernel_size - 1) * dilation

        # First convolution layer
        self.conv1 = nn.Conv1d(
            in_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
        )
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)

        # Second convolution layer
        self.conv2 = nn.Conv1d(
            out_channels,
            out_channels,
            kernel_size,
            stride=stride,
            padding=padding,
            dilation=dilation,
        )
        self.bn2 = nn.BatchNorm1d(out_channels)

        self.net = nn.Sequential(
            *(self.conv1, self.bn1, self.relu, self.dropout),
            *(self.conv2, self.bn2, self.relu, self.dropout),
        )

        # Residual connection
        self.downsample = (
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
            if in_channels != out_channels
            else None
        )

    def forward(self, x):
        out = self.net(x)
        if self.downsample is not None:
            x = self.downsample(x)
        # Slice output to match input length (in case padding added extra time steps)
        out = out[..., : x.size(-1)]
        return self.relu(out + x)


class TCN(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=3, dropout=0.2):
        super(TCN, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2**i
            in_channels = num_inputs if i == 0 else num_channels[i - 1]
            out_channels = num_channels[i]
            layers += [
                TemporalBlock(
                    in_channels,
                    out_channels,
                    kernel_size,
                    stride=1,
                    dilation=dilation_size,
                    dropout=dropout,
                ),
                # Strided convolution to downscale
                nn.Conv1d(out_channels, out_channels, 5, 3),
            ]

        self.network = nn.Sequential(*layers)
        self.linear = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        # Input shape: [batch_size, channels, timesteps]
        y = self.network(x)
        y = y[..., -1]  # Take the last time step output
        return self.linear(y)[..., 0] # Squeeze the last dim

In [7]:
from tqdm.notebook import tqdm

# Instantiate the model, loss, and optimizer
num_features = X_train.shape[-2]  # Number of features as the input channels
model = TCN(
    num_inputs=num_features,
    num_channels=[128, 128, 128],
    kernel_size=7,
    dropout=0.2,
).to(device)
criterion = nn.MSELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 256

pbar = tqdm(total=num_epochs)

for epoch in range(num_epochs):
    model.train()
    running_train_loss = 0.0
    running_train_count = 0

    # Wrap train_loader with tqdm to show progress
    for X_batch, y_batch in tqdm(
        train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}", disable=True
    ):
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch)
        loss.backward()
        optimizer.step()

        running_train_loss += loss.item()
        running_train_count += np.ones_like(y_batch.detach().cpu().numpy()).sum()

    epoch_loss = running_train_loss / running_train_count
    # print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {epoch_loss:.4f}")

    # Validation
    model.eval()
    running_val_loss = 0.0
    running_val_count = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs.squeeze(), y_batch)
            running_val_loss += loss.item()
            running_val_count += np.ones_like(y_batch.cpu().numpy()).sum()

    val_loss = running_val_loss / running_val_count
    # print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

    pbar.update()

    pbar.set_postfix(
        {
            "Train Loss": f"{epoch_loss:.4f}",
            "Val Loss": f"{val_loss:.4f}",
        }
    )

torch.save(model.state_dict(), "model.pth")

  0%|          | 0/256 [00:00<?, ?it/s]

In [3]:
def pytorch_model_to_jax(torch_model: torch.nn.Sequential):
    params = []
    act = None

    # Extract params (weights, biases) from torch layers, to be used in
    # jax.
    # Note: This propagator assumes a feed-forward relu NN.
    for m in torch_model.modules():
        if isinstance(m, torch.nn.Sequential):
            continue
        elif isinstance(m, torch.nn.ReLU):
            if act is None or act == "relu":
                act = "relu"
        elif isinstance(m, torch.nn.Linear):
            w = m.weight.data.cpu().numpy().T
            b = m.bias.data.cpu().numpy()
            params.append((w, b))
    return functools.partial(relu_nn, params)


def relu_nn(params, inputs):
    for W, b in params[:-1]:
        outputs = jnp.dot(inputs, W) + b
        inputs = jnp.maximum(outputs, 0)
    W, b = params[-1]
    return jnp.dot(inputs, W) + b


def jax_interval_to_np_range(interval: jax_verify.IntervalBound) -> np.ndarray:
    return np.vstack([interval.lower, interval.upper]).T


def np_range_to_jax_interval(input_range: np.ndarray) -> jax_verify.IntervalBound:
    return jax_verify.IntervalBound(input_range[:, 0], input_range[:, 1])


jax_model = pytorch_model_to_jax(model)

NameError: name 'torch' is not defined

In [8]:
def nominal_and_epsilon_to_range(nominal: np.ndarray, epsilon: np.ndarray | float) -> np.ndarray:
  return np.vstack([nominal-epsilon, nominal+epsilon]).T

def range_to_nominal_and_epsilon(input_range: np.ndarray) -> tuple[np.ndarray, np.ndarray]:
  nominal_input = (input_range[:, 1] + input_range[:, 0]) / 2.
  epsilon = (input_range[:, 1] - input_range[:, 0]) / 2.
  return nominal_input, epsilon


In [16]:
import torch
import numpy as np

# Extract inputs from the validation dataset and convert to NumPy
val_X_all = torch.stack([val_dataset[i][0] for i in range(len(val_dataset))]).squeeze(-1)
nominal_input = val_X_all.numpy()

# Define epsilon for input perturbation (this could be a scalar or an array)
epsilon = 0.05  # Example value, adjust as needed

# Use the provided function to convert nominal inputs and epsilon to a range
input_range = nominal_and_epsilon_to_range(nominal_input, epsilon)

In [18]:
# Example of computing bounds using IBP as implemented by jax_verify
input_bounds = np_range_to_jax_interval(input_range)
output_bounds_ibp_jax = jax_verify.interval_bound_propagation(jax_model, input_bounds)
output_range_ibp_jax = jax_interval_to_np_range(output_bounds_ibp_jax)
print(f"output bounds via IBP (jax_verify): \n{output_range_ibp_jax}")

TypeError: dot_general requires contracting dimensions to have the same shape, got (214,) and (64,).

In [ ]:
num_trials = 10000
successful_trials = 0

for _ in range(num_trials):
    perturbed_inputs = inputs + jax.random.normal(jax.random.PRNGKey(0), inputs.shape) * epsilon
    predictions = model_function(params, perturbed_inputs)
    within_bounds = jnp.logical_and(predictions >= target_lb, predictions <= target_ub)
    if jnp.all(within_bounds):
        successful_trials += 1

empirical_verification_percentage = (successful_trials / num_trials) * 100

if empirical_verification_percentage >= 99:
    print("Empirically verified: Model predicts within ±0.1 m/s for 99% of the perturbed inputs.")
else:
    print(f"Empirical verification failed. Only {empirical_verification_percentage}% of predictions are within the target range.")
